# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,294.21,61,6,9.08,ZA,1700667919
1,1,grytviken,-54.2811,-36.5092,274.60,100,100,2.58,GS,1700667920
2,2,ilulissat,69.2167,-51.1000,268.16,68,75,7.20,GL,1700667921
3,3,edinburgh of the seven seas,-37.0676,-12.3116,287.02,79,54,9.23,SH,1700667921
4,4,dalbandin,28.8947,64.4101,295.36,40,2,4.00,PK,1700667922


In [19]:


# Change the max temp to celsius
city_data_df['Max Temp (C)'] = city_data_df['Max Temp'] - 273.15

#Remake the data frame with new columns
city_data_df = city_data_df[["City_ID", "City", "Lat", "Lng", "Max Temp (C)", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]


city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp (C),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,21.06,61,6,9.08,ZA,1700667919
1,1,grytviken,-54.2811,-36.5092,1.45,100,100,2.58,GS,1700667920
2,2,ilulissat,69.2167,-51.1000,-4.99,68,75,7.20,GL,1700667921
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.87,79,54,9.23,SH,1700667921
4,4,dalbandin,28.8947,64.4101,22.21,40,2,4.00,PK,1700667922


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp (C)"] >= 21) & (city_data_df["Max Temp (C)"] <= 27) & (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna(how = "any")

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp (C),Humidity,Cloudiness,Wind Speed,Country,Date
11,11,east london,-33.0153,27.9116,25.52,73,0,4.12,ZA,1700667929
55,55,port saint john's,-31.6229,29.5448,24.70,81,0,2.93,ZA,1700667964
92,92,ubari,26.5921,12.7805,21.96,17,0,4.11,LY,1700667996
146,146,tsiombe,-25.3000,45.4833,26.44,66,0,4.15,MG,1700668040
155,155,port elizabeth,-33.9180,25.5701,26.16,47,0,3.09,ZA,1700667955
172,172,cabo san lucas,22.8909,-109.9124,26.40,50,0,0.89,MX,1700668042
177,177,ovalle,-30.5983,-71.2003,26.97,30,0,2.68,CL,1700668066
223,223,behompy,-23.2496,43.8557,26.44,68,0,2.24,MG,1700668103
230,230,kannod,22.6667,76.7333,21.27,46,0,1.82,IN,1700668109
250,250,samdari,25.8167,72.5833,22.45,22,0,1.23,IN,1700668127


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()
hotel_df = hotel_df [["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,east london,ZA,-33.0153,27.9116,73,
55,port saint john's,ZA,-31.6229,29.5448,81,
92,ubari,LY,26.5921,12.7805,17,
146,tsiombe,MG,-25.3000,45.4833,66,
155,port elizabeth,ZA,-33.9180,25.5701,47,
172,cabo san lucas,MX,22.8909,-109.9124,50,
177,ovalle,CL,-30.5983,-71.2003,30,
223,behompy,MG,-23.2496,43.8557,68,
230,kannod,IN,22.6667,76.7333,46,
250,samdari,IN,25.8167,72.5833,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
#categories = 

params = {
    "categories": "accommodation.hotel",
    "apiKey" : geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn
ubari - nearest hotel: فندق اوباري
tsiombe - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
cabo san lucas - nearest hotel: Comfort Rooms
ovalle - nearest hotel: Hotel Roxy
behompy - nearest hotel: No hotel found
kannod - nearest hotel: No hotel found
samdari - nearest hotel: No hotel found
mar del plata - nearest hotel: Gran hotel Mónaco
kruisfontein - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
harda khas - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
11,east london,ZA,-33.0153,27.9116,73,No hotel found
55,port saint john's,ZA,-31.6229,29.5448,81,Outback Inn
92,ubari,LY,26.5921,12.7805,17,فندق اوباري
146,tsiombe,MG,-25.3000,45.4833,66,No hotel found
155,port elizabeth,ZA,-33.9180,25.5701,47,Waterford Hotel
172,cabo san lucas,MX,22.8909,-109.9124,50,Comfort Rooms
177,ovalle,CL,-30.5983,-71.2003,30,Hotel Roxy
223,behompy,MG,-23.2496,43.8557,68,No hotel found
230,kannod,IN,22.6667,76.7333,46,No hotel found
250,samdari,IN,25.8167,72.5833,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    hover_cols=["City","Hotel Name","Country"]

)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)